# **신경망 모델 구성하기**

신경망은 데이터에 대한 연산을 수행하는 layer/module로 구성되어 있습니다. `torch.nn` 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공합니다. 파이토치의 모든 모듈은 `nn.Module`의 하위 클래스입니다. 신경망은 다른 모듈(layer)로 구성된 모듈입니다. 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있습니다.

이어지는 장에서는 FashionMNIST데이터셋의 이미지들을 분류하는 신경망을 구성하곘습니다.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

### 클래스 정의하기

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
# NeuralNetwork의 instance를 생성하고 이를 device로 이동시킨 뒤, 구조를 출력합니다.
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


모델을 사용하기 위해 입력 데이터를 전달합니다. 이는 일부 `백그라운드 연산들`과 함께 모델의 forward를 실행합니다. **model.forward()를 직접 호출하면 안 됩니다.**

모델에 입력을 호출하면 각 class에 대한 raw 예측값이 있는 10차원 텐서가 반환됩니다. raw 예측값을 `nn.Softmax` 모듈의 인스턴스에 통과시켜 예측 확률을 얻습니다.

In [42]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4])


### 모델 layer

FashionMNIST 모델의 계층들을 살펴보겠습니다. 이를 설명하기 위헤, 28 x 28 크기의 이미지 3개로 구성된 미니배치를 가져와서 신경망을 통과할 때 어떤 일이 발생하는지 보겠습니다.

In [43]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


**nn.Flatten**

`nn.Flatten`은 계층을 초기화하여 각 28 x 28의 2D 이미지를 784 픽셀값을 갖는 연속된 배열로 반환합니다. (dim=0의 미니배치 차원은 유지됩니다.)

In [44]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


**nn.Linear**

`nn.Linear`는 저장된 가중치와 편향을 사용하여 입력에 선형 변환을 적용하는 모듈입니다.

In [45]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


**nn.ReLU**

nn.ReLU같은 비선형 활성화 함수는 모델의 입력과 출력 사이에 복잡한 관계(mapping)를 만듭니다. 비선형 활성화 함수는 선형 변환 후에 적용되어 비선형성을 가지게 만들고, 신경망이 다양한 현상을 학습할 수 있도록 돕습니다. 

이 모델에서는 `nn.ReLU`를 선형 계층들 사이에 사용하지만, 모델을 만들 때는 비선형성을 가진 다른 활성화를 도입할 수도 있습니다.

In [46]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1150, -0.0127, -0.5631, -0.1334, -0.1226,  0.2719,  0.6505, -0.0818,
         -0.3713, -0.1237, -0.3054,  0.1297,  0.1276,  0.5605, -0.4894, -0.2783,
          0.4900,  0.5113,  0.2090, -0.1941],
        [ 0.0420,  0.2975, -0.8358, -0.2193,  0.0226,  0.2357,  0.3952, -0.5156,
         -0.5239, -0.0562, -0.3520,  0.1118,  0.4400,  0.5300, -0.5167, -0.2825,
          0.4869,  0.5671,  0.0068,  0.1407],
        [ 0.0564,  0.3625, -0.9141, -0.3772, -0.2132,  0.2234,  0.7288, -0.4724,
         -0.6680, -0.0745, -0.7985,  0.0420,  0.4115,  0.4512, -0.5639, -0.2956,
          0.3720,  0.2536,  0.2319, -0.1154]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1150, 0.0000, 0.0000, 0.0000, 0.0000, 0.2719, 0.6505, 0.0000, 0.0000,
         0.0000, 0.0000, 0.1297, 0.1276, 0.5605, 0.0000, 0.0000, 0.4900, 0.5113,
         0.2090, 0.0000],
        [0.0420, 0.2975, 0.0000, 0.0000, 0.0226, 0.2357, 0.3952, 0.0000, 0.0000,
         0.0000, 0.0000, 0.1118, 0.4400, 0.5300, 0.00

**nn.Sequential**

`nn.Sequential`은 순서를 갖는 모듈의 컨테이너입니다. 데이터는 정의된 것과 같은 순서로 모든 모듈을 통해 전달됩니다. Sequential container를 사용하여 아래의 `seq_modules`와 같은 신경망을 빠르게 만들 수 있습니다.

In [47]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10),
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

**nn.Softmax**

신경망의 마지막 선형 계층은 `nn.Softmax` 모듈에 전달될 logits를 반환합니다. logits는 모델의 각 class에 대한 예측 확률을 나타내도록 0 ~ 1 범위로 비례하여 조정됩니다. dim 매개변수는 값의 합이 1이 되는 차원을 나타냅니다.

In [48]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

### 모델 매개변수

신경망 내부의 많은 계층들을 매개변수화됩니다. 즉, 학습 중에 최적화되는 가중치와 편향과 연관지어집니다. nn.Module을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적되며, 모델의 parameters() 및 named_parameters() 메소드로 모든 매개변수에 접근할 수 있게 됩니다.

이 예제에서는 각 매개변수들을 순회하며, 매개변수의 크기와 값을 출력합니다.

In [49]:
print("Model structure: ", model, '\n\n')

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0194, -0.0240,  0.0140,  ...,  0.0097,  0.0309,  0.0056],
        [-0.0041,  0.0050, -0.0172,  ...,  0.0241, -0.0280, -0.0092]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0347, -0.0077], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0173, -0.0366, -0.0185,  ..., -0.0090,  0.0104,  0.0025],
        [-0.0368,  0.0421,  0.0144,  ..., -0.0240, -0.0233,  0.0395]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | S